In [1]:
import sys
sys.path.append("../")

In [20]:
import os

import numpy as np
import pandas as pd
import joblib

from agents.matchmanager import buildMatchManager
from core.game.state import vectorStateInfo, vectorState, vectorActionInfo, vectorAction

from sklearn.ensemble import RandomForestRegressor

from datetime import datetime
from multiprocessing import Pool

In [3]:
raw = pd.read_pickle('../data.2021-02-21.scenarioJunction.pkl.gz')\
        .drop(['meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'], axis=1, errors="ignore")

In [4]:
df = raw.dropna(axis=1, how='all')

In [7]:
df_red = df[df['action_team'] == 'red'].copy().drop('action_team', axis=1)
df_blue = df[df['action_team'] == 'blue'].copy().drop('action_team', axis=1)

In [8]:
df_red['label'] = df_red['winner'].apply(lambda x: 1 if x == 'red' else -1)
df_blue['label'] = df_blue['winner'].apply(lambda x: 1 if x == 'blue' else -1)

In [10]:
df_red_move=df_red.loc[((df_red['action_type_Move'] == True)|(df_red['action_type_LoadInto'] == True))].copy()
df_red_attack=df_red.loc[((df_red['action_type_Attack'] == True)| (df_red['action_type_AttackGround'] == True) |(df_red['action_type_AttackRespond'] == True))].copy()
df_red_pass=df_red.loc[((df_red['action_type_Pass'] == True)| (df_red['action_type_PassFigure'] == True) |(df_red['action_type_PassTeam'] == True) | (df_red['action_type_PassRespond'] == True))].copy()

In [14]:
df_blue_move=df_blue.loc[((df_blue['action_type_Move'] == True)|(df_blue['action_type_LoadInto'] == True))].copy()
df_blue_attack=df_blue.loc[((df_blue['action_type_Attack'] == True)| (df_blue['action_type_AttackGround'] == True) |(df_blue['action_type_AttackRespond'] == True))].copy()
df_blue_pass=df_blue.loc[((df_blue['action_type_Pass'] == True)| (df_blue['action_type_PassFigure'] == True) |(df_blue['action_type_PassTeam'] == True) | (df_blue['action_type_PassRespond'] == True))].copy()

In [16]:
X_red_move = df_red_move.drop(['winner', 'label'], axis=1)
y_red_move = df_red_move['label']

X_blue_move = df_blue_move.drop(['winner', 'label'], axis=1)
y_blue_move = df_blue_move['label']

In [17]:
X_red_attack = df_red_attack.drop(['winner', 'label'], axis=1)
y_red_attack  = df_red_attack['label']

X_blue_attack  = df_blue_attack.drop(['winner', 'label'], axis=1)
y_blue_attack  = df_blue_attack['label']

In [18]:
X_red_pass = df_red_pass.drop(['winner', 'label'], axis=1)
y_red_pass  = df_red_pass['label']

X_blue_pass  = df_blue_pass.drop(['winner', 'label'], axis=1)
y_blue_pass  = df_blue_pass['label']

In [19]:
X = pd.concat([X_red_move, X_blue_move,X_red_attack,X_blue_attack,X_red_pass,X_blue_pass])
y = pd.concat([y_red_move, y_blue_move,y_red_attack,y_blue_attack,y_red_pass,y_blue_pass])

In [21]:
rfr_red_move = RandomForestRegressor()
rfr_blue_move = RandomForestRegressor()
rfr_red_attack = RandomForestRegressor()
rfr_blue_attack = RandomForestRegressor()
rfr_red_pass = RandomForestRegressor()
rfr_blue_pass = RandomForestRegressor()
rfr = RandomForestRegressor()

rfr_red_move.fit(X_red_move, y_red_move)
rfr_red_attack.fit(X_red_attack, y_red_attack)
rfr_red_pass.fit(X_red_pass, y_red_pass)

rfr_blue_move.fit(X_blue_move, y_blue_move)
rfr_blue_attack.fit(X_blue_attack, y_blue_attack)
rfr_blue_pass.fit(X_blue_pass, y_blue_pass)


rfr.fit(X, y)

joblib.dump(rfr_red_move, '../models/Junction_RandomForestRegressor_red_move_20210221.joblib')
joblib.dump(rfr_red_attack, '../models/Junction_RandomForestRegressor_red_attack_20210221.joblib')
joblib.dump(rfr_red_pass, '../models/Junction_RandomForestRegressor_red_pass_20210221.joblib')


joblib.dump(rfr_blue_move, '../models/Junction_RandomForestRegressor_blue_move_20210221.joblib')
joblib.dump(rfr_blue_attack, '../models/Junction_RandomForestRegressor_blue_attack_20210221.joblib')
joblib.dump(rfr_blue_pass, '../models/Junction_RandomForestRegressor_blue_pass_20210221.joblib')

joblib.dump(rfr, '../models/Junction_RandomForestRegressor_20210221.joblib')

['../models/Junction_RandomForestRegressor_20210221.joblib']